In [73]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
import tensorflow as tf
#import tensorflow_addons as tfa

In [74]:
img_size = (224, 224)

# train data
cwd = os.getcwd()
epochs = 100
df = pd.read_csv(cwd + '/state/train.csv')
X = list(df['image'][i] for i in range(len(df)))
y = list(df['emotion'][i] for i in range(len(df)))

In [75]:
weight = dict(enumerate(class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)))
print(weight)

{0: 1.8591824644549764, 1: 2.264939376443418, 2: 1.8262918994413408, 3: 1.857421875, 4: 0.4784577387486279, 5: 0.41608771743742046, 6: 1.4797717842323652, 7: 1.3063186813186813}


In [76]:

def createModel():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (img_size[0],img_size[1],3)) ,
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(), 
        tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
        tf.keras.layers.Dropout(0.1,seed = 2019),
        tf.keras.layers.Dense(400,activation ="relu"),
        tf.keras.layers.Dropout(0.3,seed = 2019),
        tf.keras.layers.Dense(300,activation="relu"),
        tf.keras.layers.Dropout(0.4,seed = 2019),
        tf.keras.layers.Dense(200,activation ="relu"),
        tf.keras.layers.Dropout(0.2,seed = 2019),
        tf.keras.layers.Dense(5,activation = "softmax")   #Adding the Output Layer
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(amsgrad=True),  # tf.keras.optimizers.Adam(amsgrad=True),  # tf.keras.optimizers.SGD(nesterov=True), # tfa.optimizers.RectifiedAdam(),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits=True),
            metrics=['accuracy'])
    return model


In [77]:
# es_callback = tf.keras.callbacks.EarlyStopping(
#     monitor='val_accuracy', patience=epochs*0.05, restore_best_weights=True)
# cp_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath="model/cp-{epoch:04d}.ckpt",
#     verbose=0,
#     save_weights_only=True,
#     monitor='val_accuracy',
#     save_best_only=True)
# tb_callback = tf.keras.callbacks.TensorBoard(
#     log_dir="./logs",
#             update_freq="epoch")

In [78]:
model = createModel()
print(model.summary())


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 111, 111, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 54, 54, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 26, 26, 64)      

In [79]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
adam=Adam(lr=0.001)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])


C:\Python39\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [85]:
import os, cv2, shutil
df_train = pd.read_csv(cwd + "\\state\\train.csv")
df_test = pd.read_csv(cwd + "\\state\\test.csv")
output_train_dir = cwd + "\\train"
root_image_dir = cwd + "\\cleaned_images\\"
dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']
dir_test = cwd + "\\test\\"
dir_train = cwd + "\\train\\"
if(os.path.exists(dir_train) or os.path.exists(dir_test)):
    shutil.rmtree(dir_train)
    shutil.rmtree(dir_test)
for emotion in dictionary:
    os.makedirs(dir_test + "\\" + emotion)
    os.makedirs(dir_train + "\\" + emotion)

def isInDir(image_dir):
    return len(image_dir.split("/")) >= 2
for index, row in df_test.iterrows():
    if(isInDir(row['image']) == False): continue
    output_test_dir = dir_test + row['image']
    image = cv2.imread(root_image_dir + row['image'])
    cv2.imwrite(output_test_dir, image)
for index, row in df_train.iterrows():
    if(isInDir(row['image']) == False): continue
    output_train_dir = dir_train + row['image']
    image = cv2.imread(root_image_dir + row['image'])
    cv2.imwrite(output_train_dir, image)


In [86]:
bs = 30
train_dir = cwd + "\\train"
test_dir = cwd + "\\test"
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(img_size[0],img_size[1]))
validation_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(img_size[0],img_size[1]))
dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    labels='inferred',
    label_mode='int',
    image_size=img_size,
    class_names=dictionary,
    color_mode='rgb',
    batch_size=bs)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=img_size,
    class_names=dictionary,
    color_mode='rgb',
    batch_size=bs)

Found 20279 images belonging to 8 classes.
Found 2499 images belonging to 8 classes.
Found 20279 files belonging to 8 classes.
Found 2499 files belonging to 8 classes.


In [87]:

es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=epochs*0.05, restore_best_weights=True)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model/cp-{epoch:04d}.ckpt",
    verbose=0,
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=True)
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir="./logs",
            update_freq="epoch")

In [90]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=150 // bs,
                    epochs=30,
                    validation_steps=50 // bs,
                    verbose=2)

Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Pongsakorn\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\Pongsakorn\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Pongsakorn\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Pongsakorn\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", line 199, in start
     